<a href="https://colab.research.google.com/github/HedersonSantos/age-related/blob/main/auto_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

for _ in range(3):
    try:
        import autosklearn.classification
        break
    except:
        pass
else:
    raise ImportError("failed to import from autosklearn")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.8ubuntu1.1).
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  swig swig4.0
0 upgraded, 2 newly installed, 0 to remove and 24 not upgraded.
Need to get 1,086 kB of archives.
After this operation, 5,413 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig4.0 amd64 4.0.1-5build1 [1,081 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 swig all 4.0.1-5build1 [5,528 B]
Fetched 1,086 kB in 0s (6,671 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
deb

In [ ]:
!wget https://raw.githubusercontent.com/HedersonSantos/age-related/main/tree_features .csv


--2023-05-22 23:05:21--  https://raw.githubusercontent.com/HedersonSantos/age-related/main/tree_features.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111303 (109K) [text/plain]
Saving to: ‘tree_features.csv.1’

tree_features.csv.1 100%[===================>] 108.69K  --.-KB/s    in 0.02s   

2023-05-22 23:05:21 (4.29 MB/s) - ‘tree_features.csv.1’ saved [111303/111303]



In [ ]:
df = pd.read_csv('tree_features.csv', header=0)
df.head(10)

,AB,AF,BC,BQ,CC,CR,DA,DE,DH,DI,DU,EE,EH,FD,FE,FI,FL,FR,GL
0,0.209377,3109.03329,5.555634,152.707705,0.563481,0.069225,69.08340,295.570575,0.284232,89.245560,5.310690,1.987283,0.949104,10.265073,9028.291921,3.583450,7.298162,1.73855,0.120343
1,0.145282,978.76416,1.229900,14.754720,0.484710,1.117800,70.79836,178.553100,0.363489,110.581815,0.005518,0.858603,0.003042,0.296850,6785.003474,10.358927,0.173229,0.49706,21.978000
2,0.470030,2635.10654,1.229900,219.320160,0.495852,0.700350,70.81970,321.426625,0.210441,120.056438,1.289739,8.146651,0.377208,8.745201,8338.906181,11.626917,7.709560,0.97556,0.196941
3,0.252107,3819.65177,1.229900,11.050410,0.717882,0.636075,47.27586,196.607985,0.292431,139.824570,2.655345,3.813326,0.614484,7.884336,10965.766040,14.852022,6.122162,0.49706,0.155829
4,0.380297,3733.04844,102.151980,149.717165,0.536467,0.693150,74.06532,200.178160,0.207708,97.920120,1.144902,3.490846,0.164268,4.274640,16198.049590,13.666727,8.153058,48.50134,0.096614
5,0.209377,2615.81430,1.229900,16.526120,0.639460,0.857625,55.22404,135.489250,0.478275,135.317865,0.005518,2.394414,0.003042,0.296850,8517.278846,10.981896,0.173229,0.49706,21.978000
6,0.348249,1733.65412,1.229900,344.644105,0.946323,0.610950,19.21570,107.907985,0.460510,176.625563,1.006962,2.620150,0.097344,1.460502,3903.806766,10.777915,4.408484,0.86130,0.065096
7,0.269199,966.45483,1.229900,6.199900,0.771909,1.109625,63.21684,326.225295,0.325227,83.769368,2.117379,3.277203,0.292032,4.518057,18090.349450,10.342388,6.591896,0.49706,0.092873
8,0.346113,3238.43674,3.626448,61.642115,0.855496,1.050375,76.77356,231.134460,0.330693,131.349555,0.613833,1.342323,0.066924,1.389258,3380.026318,11.450501,4.762291,1.18262,0.073416
9,0.324748,5188.68207,1.229900,107.276505,0.718271,0.711712,31.17774,403.247940,0.351191,129.175267,0.005518,3.658132,0.003042,0.296850,3142.390734,12.329825,0.173229,1.57151,21.978000


In [ ]:
X_train = df[:-1]
y_train = df['Class']

In [ ]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=600,
    per_run_time_limit=60,
    tmp_folder="/tmp/autosklearn_classification_example_tmp",
)
automl.fit(X_train, y_train, dataset_name="breast_cancer")

In [ ]:
print(automl.leaderboard())   

In [ ]:
print(automl.show_models(), indent=4)

In [ ]:
predictions = automl.predict(X_test)
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))